In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Flash Crash Detection code base in the drift-burst hypothesis

In [ ]:
import pandas as pd
import numpy as np
import pickle #To save the results 


In [ ]:
def resample_quote(clean_data,interval='s'): 
  clean_data = clean_data.set_index(pd.DatetimeIndex(clean_data['TIME_M']))
  clean_data= clean_data.resample(interval).agg({'BID':'mean', 'ASK':'mean',
                                                 'ASKSIZ':'sum','BIDSIZ':'sum',
                                                 'SPREAD':'mean','MID_QUOTE':'mean',
                                                 'NUM_TICKS':'sum' }) #,'VOLAT':'sum' But some seconds only have 1 obs
                                                
  clean_data = clean_data.dropna()
  UPDATE_BID = clean_data['BID'] - clean_data['BID'].shift(1)
  UPDATE_ASK = clean_data['ASK'] - clean_data['ASK'].shift(1)
  return clean_data[(UPDATE_BID!=0)&(UPDATE_ASK!=0)].copy() #we consider only prices that changed the next period for the t-stat

### Instantaneous drift estimator

In [ ]:
def drift (h, t, p):
    """Function to estimate a local drift by using the kernel estimator
    
    Parameters
    ----------
    h: Float
       Bandwidth in seconds
       
    t: Timestamp dataframe, list, array
       Vector of time series
       
    p: Dataframe, list, array
       Vector of stock prices
       
    Returns
    -------
    List
        Local Estimate Drift
        
        """
    s = np.timedelta64(h, 's')
    T = len(t) # n + 1
    #t = pd.to_datetime(pd.Series(t)) to work with pandas series but it is slower
    t = np.array(pd.to_datetime(t))   
    p = np.array(p)
    p_t = np.log(p)
    r_t = p_t[1:] - p_t[0:-1] #p_t - p_t.shift(1) funciona con dataframe
    mu = []
    for j in range(1,T):
        s0 = (t>=t[j]-s) 
        s1 = (t<=t[j])
        s_1 = (t<t[j])
        t_n = t[s0 & s1] #.copy() #it takes all the data "s" seconds before the j time
        r_n = r_t[np.where(s0 & s_1)] #r_t is a N-1 matrix
        K =  ((t_n[:-1] - t_n[-1])/h)/np.timedelta64(1,'s') #another option is (t_n.iloc[i-1] - t_n).dt.total_seconds()/h pero es lento
        Kr = np.exp(K) * r_n
        #Kr = Kr*(K<=0) We want to avoid look ahead bias, therefor we just took lagged values, but in this case all values <=0
        mu_i = np.sum(Kr)/h
        mu.append(mu_i)
    return mu

 ### Spot volatility estimator

In [ ]:
def s_volat (h, t, p):
    """Function to estimate a local spot volatility by using the kernel estimator
    
    Parameters
    ----------
    h: Float
       Bandwidth in seconds
       
    t: Timestamp dataframe, list, array
       Vector of time series
       
    p: Dataframe, list, array
       Vector of stock prices
       
    Returns
    -------
    List
        Local Estimate Spot Volatility
        
        """
    s = np.timedelta64(h, 's')
    T = len(t)
    #t = pd.to_datetime(pd.Series(t)) to work with pandas series but it is slower
    t = pd.to_datetime(t).values #values convert it into numpy array    
    p = np.array(p)
    p_t = np.log(p)
    r_t = p_t[1:] - p_t[0:-1] #p_t - p_t.shift(1) funciona con dataframe
    sigma = []
    for j in range(1,T):
        s0 = (t>=t[j]-s) 
        s1 = (t<=t[j])
        s_1 = (t<t[j])
        t_n = t[s0 & s1] #.copy() #it takes all the data "s" seconds before the j time
        r_n = r_t[np.where(s0 & s_1)] #r_t is a N-1 matrix
        K =  ((t_n[:-1] - t_n[-1])/h)/np.timedelta64(1,'s')
        Kr = np.exp(K) * r_n**2  
        #Kr = Kr*(K>0)
        sigma_i = np.sqrt(np.sum(Kr)/h)
        sigma.append(sigma_i)
    return sigma

## Robust version
According to the authors the prior test loss its power with the high frequency data due to the microstructure of the market generates noise 

### Pre-averaging 

In [ ]:
def pre_avg(v,k=3):
    """Function to estimate a pre-averaged increment of an stochastich process
        This function is necesary for the robust estimation of the drift burst
    
    Parameters
    ----------
    v: Dataframe, list, array
       Vector of stochastic process
       
    k: Pre-averaging window (Default 3)
    
    Returns
    -------
    List
        Pre-average increment of the stochastic process
    """
    N = len(v) 
    v = np.array(v)
    v = np.log(v)
    v_hat = []
    
    for i in range(0,N):
        vg = []
        if i < N-k+1:
            for j in range(1,k): #Since range function is [0,k_n), so it takes k_n-1
                v_ = v[i+j] - v[i+j-1]
                g_j = min(j/k,1-j/k) #function g(i/k_n) in this case min
                vg.append(v_*g_j)
            v_hat.append(sum(vg))
        else:
            v_hat.append([])
        
    return v_hat      
    

### Parzen smoot kernel

In [ ]:
def parz(L):
    """Parzen smooth Kernel suggested in Christensen et. al(2017)
    Parameters
    ----------
    L: Float
       Lag-lenght ratio (L/Ln) where L is the current lag and Ln the total lag lenght
       
    Returns
    -------
    Float
        Parzen smooth kerlnel for the given point
    """
    L_ = np.abs(L)
    if 0 <= L_ <= 1/2:
        parzen = 1 - 6*L**2 +6*L_**3
    elif 1/2 < L_ <= 1:
        parzen = 2*(1-L_)**3
    else:
        parzen = 0
        
    return parzen

### Robust drift estimator

In [ ]:
def robust_drift (h, t, p, k=3):
    """Function to estimate a local drift by using the robust kernel estimator
    
    Parameters
    ----------
    h: Floatlen
       Bandwidth in seconds
       
    t: Timestamp dataframe, list, array
       Vector of time series
       
    p: Dataframe, list, array
       Vector of stock prices
       
    k: Float
       Pre-average window (Default 3)
       
    Returns
    -------
    List
        Local Estimate Drift
        
        """
    s = np.timedelta64(h, 's')
    T = len(t)
    t = pd.to_datetime(t).values #values convert it into numpy array    
    p = np.array(p)
    p_t = np.log(p)
    r_t = np.array(pre_avg(p,k))
    mu = []
    for j in range(0,T): #in the document goes to 
        s0 = (t>=t[j]-s) 
        s1 = (t<=t[j])
        #s_1 = (t<t[j])
        t_n = t[s0 & s1]
        if len(t_n)>k-1:
            r_n = r_t[np.where((s0 & s1))][0:-k+1]
            K =  ((t_n[0:-k+1] - t_n[-1])/h)/np.timedelta64(1,'s') #another option is (t_n.iloc[i-1] - t_n).dt.total_seconds()/h pero es lento
            Kr = np.exp(K) * r_n  
            mu_i = np.sum(Kr)/h
            mu.append(mu_i)
        
    return mu

## Robust volatility estimator

In [ ]:
def robust_s(h, t, p, k=3):
    """Function to estimate a local spot volatility by using the robust kernel estimator
    
    Parameters
    ----------
    h: Float
       Bandwidth in seconds
       
    t: Timestamp dataframe, list, array
       Vector of time series
       
    p: Dataframe, list, array
       Vector of stock prices
       
    k: Float
       Pre-average window (Default 3)
       
    Returns
    -------
    List
        Local Estimate Spot Volatility
        
        """
    s = np.timedelta64(h, 's')
    T = len(t)
    t = pd.to_datetime(t).values #values convert it into numpy array    
    p = np.array(p)
    p_t = np.log(p)
    r_t = np.array(pre_avg(p,k))
    l_n = 2*(k - 1)+10 #lag selection defined in christensen
    s_robust = []
    for j in range(0,T):
        s0 = (t>=t[j]-s) 
        s1 = (t<=t[j])
        t_n = t[s0 & s1] #.copy() #it takes all the data "s" seconds before the j time
        s_i=[]
        if len(t_n)>k-1:
            r_n = r_t[np.where((s0 & s1))][0:-k+1]
            K =  ((t_n[0:-k+1] - t_n[-1])/h)/np.timedelta64(1,'s') #another option is (t_n.iloc[i-1] - t_n).dt.total_seconds()/h pero es lento
            Kr = np.exp(K) * r_n 
            s_i = np.sum(Kr**2)/h
            w_i =  []
            
            for l in range(1,l_n+1):
                w_l = np.sum(parz(l/l_n) * Kr[0:-l] * Kr[l:])
                w_i.append(w_l)
                w = np.sum(w_i)/h
                s_hat = s_i + 2*w
            s_robust.append(s_hat) #order and indent are important
        
    return s_robust
    

### T-test statistic

In [ ]:
def t_stat(h_mu, h_s, t, p, K_2 = 0.5, robust = False):
    """Function to estimate the local t-stat using kernel estimation
    
    Parameters
    ----------
    h_mu: Float
          Bandwidth in seconds for the drift
       
    h_s: Float
         Bandwidth in seconds for the spot volatility
       
    t: Timestamp dataframe, list, array
       Vector of time series
       
    p: Dataframe, list, array
       Vector of stock prices
       
    K_2: Float
         Kernel dependent constant 
       
    Returns
    -------
    List
        Local t_stat for each data point
    """
    if robust == False:
        mu = drift(h_mu,t,p)
        s = s_volat(h_s,t,p)
        T = np.sqrt(h_mu/K_2) * np.divide(mu,s)
        
    else:
        mu = robust_drift(h_mu,t,p)
        s = robust_s(h_s,t,p)
        T = np.sqrt(h_mu/K_2) * np.divide(mu,np.sqrt(s))
        return T
    
    

## Data implementation

### Trade data Apple 06.05.2010

In [ ]:
#Data analysis for trade data Apple 06.05.2010

data = data.set_index(pd.DatetimeIndex(data['timestamp']))
data_second= data.resample('s').agg({'PRICE': 'mean', 'SIZE': 'sum', 'timestamp':'last'}).copy()
data_second = data_second.dropna()
data_second['update'] = data_second['PRICE'] - data_second['PRICE'].shift(1)
data_second = data_second[data_second['update']!=0].copy()
p = data_second['PRICE']
t = data_second['timestamp']
T = t_stat(300,1500,t,p,robust=True)

#to save and load the results
import pickle
output = open('t_test_06_05_10.pkl', 'wb')
pickle.dump(T, output)
output.close()

pkl_file = open('t_test_06_05_10.pkl', 'rb')
T = pickle.load(pkl_file)
pkl_file.close()


data_ommited = data.groupby(['DATE']).apply(lambda x: x[0:-2]) #Data ommiting the last 2 variables each day. They are not included in the pre avg
data_ommited = data_ommited.set_index(pd.DatetimeIndex(data_ommited['timestamp']))
data_ommited['timestamp'] = pd.to_datetime(data_ommited['timestamp']) #We have to set it up earlier in the code, no here
crit_value = np.abs(T)>2
data_ommited = data_ommited[crit_value]


### Quote data Apple 05.06.2010

In [ ]:
def resample_quote(clean_data,interval='s'): 
  clean_data = clean_data.set_index(pd.DatetimeIndex(clean_data['TIME_M']))
  clean_data= clean_data.resample(interval).agg({'BID':'mean', 'ASK':'mean',
                                                 'ASKSIZ':'sum','BIDSIZ':'sum',
                                                 'SPREAD':'mean','MID_QUOTE':'mean',
                                                 'NUM_TICKS':'sum'})
                                                
  clean_data = clean_data.dropna()
  UPDATE_BID = clean_data['BID'] - clean_data['BID'].shift(1)
  UPDATE_ASK = clean_data['ASK'] - clean_data['ASK'].shift(1)
  return clean_data[(UPDATE_BID!=0)&(UPDATE_ASK!=0)].copy() #we consider only prices that changed the next period for the t-stat

In [ ]:
data = pd.read_csv('/content/drive/My Drive/Tesis Konstanz/Clean data/data/apple_cleaned_quote.csv')
data['NUM_TICKS'] = 1
data = resample_quote(data)
p = data['MID_QUOTE']
t = data.index
T_apple = t_stat(300,1500,t,p,robust=True)

In [ ]:
output = open('/content/drive/My Drive/Tesis Konstanz/Clean data/t_test_apple_06_05_10.pkl', 'wb')
pickle.dump(T_apple, output)
output.close()

pkl_file = open('/content/drive/My Drive/Tesis Konstanz/Clean data/t_test_apple_06_05_10.pkl', 'rb')
T_apple = pickle.load(pkl_file)
pkl_file.close()

In [ ]:
#Optimal threshold

#One simple way might be look at the percentiles and boxplot, but it has limitations since it depends on the sample
np.percentile(T, [1, 50, 99])
#The lowest 1% percentile correspond to -4.58 and the top 99% to 2.9
#setting a threshold at 4.58 is a bit above Christensen result

import matplotlib.pyplot as plt
plt.style.use('ggplot')

fig, ax = plt.subplots()
ax.boxplot((T), vert=False, showmeans=True, meanline=True,
           patch_artist=True,
           medianprops={'linewidth': 2, 'color': 'purple'},
           meanprops={'linewidth': 2, 'color': 'red'})
plt.show()


### Quote data S&P 500 (SPY DTF) 06.05.2010

In [ ]:
data = pd.read_csv('/content/drive/My Drive/Tesis Konstanz/Clean data/data/s&p_cleaned_quote1.csv')
data['NUM_TICKS'] = 1
data = resample_quote(data)
p = data['MID_QUOTE']
t = data.index
T_sp1 = t_stat(300,1500,t,p,robust=True)


In [ ]:
output = open('/content/drive/My Drive/Tesis Konstanz/Clean data/t_test_sp_06_05_10.pkl', 'wb')
pickle.dump(T_sp1, output)
output.close()

pkl_file = open('/content/drive/My Drive/Tesis Konstanz/Clean data/t_test_sp_06_05_10.pkl', 'rb')
T_sp1 = pickle.load(pkl_file)
pkl_file.close()

In [ ]:
#Optimal threshold by percentile and Box-Plot
np.percentile(T_sp1, [1, 50, 99])
#lowest 1% -4.52 highest 99% 3.22
import matplotlib.pyplot as plt
plt.style.use('ggplot')

fig, ax = plt.subplots()
ax.boxplot((T_sp1), vert=False, showmeans=True, meanline=True,
           patch_artist=True,
           medianprops={'linewidth': 2, 'color': 'purple'},
           meanprops={'linewidth': 2, 'color': 'red'})
plt.show()


### Quote data Dow Jones (DIA DTF) 05.02.2018

In [ ]:
data = pd.read_csv('/content/drive/My Drive/Tesis Konstanz/Clean data/data/djia_cleaned_quote.csv')
data['NUM_TICKS'] = 1
data = resample_quote(data)
p = data['MID_QUOTE']
t = data.index
T_dj = t_stat(300,1500,t,p,robust=True)

In [ ]:
output = open('/content/drive/My Drive/Tesis Konstanz/Clean data/t_test_dj_05_02_18.pkl', 'wb')
pickle.dump(T_dj, output)
output.close()

pkl_file = open('/content/drive/My Drive/Tesis Konstanz/Clean data/t_test_dj_05_02_18.pkl', 'rb')
T_dj = pickle.load(pkl_file)
pkl_file.close()

In [ ]:
#Optimal threshold by percentile and Box-Plot
np.percentile(T_dj, [1, 50, 99])
#lowest 1% -4.72, highest 99% 3.68

import matplotlib.pyplot as plt
plt.style.use('ggplot')

fig, ax = plt.subplots()
ax.boxplot((T_dj), vert=False, showmeans=True, meanline=True,
           patch_artist=True,
           medianprops={'linewidth': 2, 'color': 'purple'},
           meanprops={'linewidth': 2, 'color': 'red'})
plt.show()

In [ ]:
#data = pd.read_csv("clean_data1.csv", nrows=500)
#data = pd.read_csv("clean_data1.csv", skiprows=range(1,1000), nrows=500) from row 1000 to 1500 with header
#For tomorrow:
#Modify the code too have just the interval between [t-hn,t] as christensen pg. 9 suggest

# Appendix (Robustness)

In [ ]:
#Optimal threshold by isolation forest
from sklearn.ensemble import IsolationForest
alg=IsolationForest(n_estimators=100, max_samples='auto', contamination=0.01, \
                    max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)

#Default Iforest with 100 trees and max 256 samples. 
#For replication purposes I set a seed at the integer 42 

In [ ]:
dow = pd.read_csv('/content/drive/My Drive/Tesis Konstanz/Clean data/data/djia_cleaned_quote.csv')
dow['NUM_TICKS'] = 1
s_p = pd.read_csv('/content/drive/My Drive/Tesis Konstanz/Clean data/data/s&p_cleaned_quote1.csv')
s_p['NUM_TICKS'] = 1
apple = pd.read_csv('/content/drive/My Drive/Tesis Konstanz/Clean data/data/apple_cleaned_quote.csv')
apple['NUM_TICKS'] = 1


#Resamplig data
dow_sec = resample_quote(dow)
window_stop_row = dow_sec[dow_sec.index < '2018-02-05 09:55:01'].iloc[-1]
print('The index at 9:55 am is', dow_sec.index.get_loc(window_stop_row.name))
s_p_sec = resample_quote(s_p)
window_stop_row = s_p_sec[s_p_sec.index < '2010-05-06 09:55:01'].iloc[-1]
print('The index at 9:55 am is',s_p_sec.index.get_loc(window_stop_row.name))
apple_sec = resample_quote(apple)
window_stop_row = apple_sec[apple_sec.index < '2010-05-06 09:55:01'].iloc[-1]
print('The index at 9:55 am is',apple_sec.index.get_loc(window_stop_row.name))

The index at 9:55 am is 1500
The index at 9:55 am is 1474
The index at 9:55 am is 1479


In [ ]:
#Quote data apple 
p = apple_sec['MID_QUOTE']
t = apple_sec.index
T_apple_10min = t_stat(600,3000,t,p,robust=True)
T_apple_15min = t_stat(900,4500,t,p,robust=True)

In [ ]:
output = open('/content/drive/My Drive/Tesis Konstanz/Clean data/t_test_apple_10min.pkl', 'wb')
pickle.dump(T_apple_10min, output)
output.close()
output = open('/content/drive/My Drive/Tesis Konstanz/Clean data/t_test_apple_15min.pkl', 'wb')
pickle.dump(T_apple_15min, output)
output.close()

In [ ]:
p = s_p_sec['MID_QUOTE']
t = s_p_sec.index
T_sp_10min = t_stat(600,3000,t,p,robust=True)
T_sp_15min = t_stat(900,4500,t,p,robust=True)

In [ ]:
output = open('/content/drive/My Drive/Tesis Konstanz/Clean data/t_test_sp_10min.pkl', 'wb')
pickle.dump(T_sp_10min, output)
output.close()
output = open('/content/drive/My Drive/Tesis Konstanz/Clean data/t_test_sp_15min.pkl', 'wb')
pickle.dump(T_sp_15min, output)
output.close()

In [ ]:
p = dow_sec['MID_QUOTE']
t = dow_sec.index
T_dj_10min = t_stat(600,3000,t,p,robust=True)
T_dj_15min = t_stat(900,4500,t,p,robust=True)

In [ ]:
output = open('/content/drive/My Drive/Tesis Konstanz/Clean data/t_test_dj_10min.pkl', 'wb')
pickle.dump(T_dj_10min, output)
output.close()
output = open('/content/drive/My Drive/Tesis Konstanz/Clean data/t_test_dj_15min.pkl', 'wb')
pickle.dump(T_dj_15min, output)
output.close()

## Loading pre estimated drift bursts

In [ ]:
pkl_file = open('/content/drive/My Drive/Tesis Konstanz/Clean data/t_test_apple_10min.pkl', 'rb')
T_apple_10min = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('/content/drive/My Drive/Tesis Konstanz/Clean data/t_test_apple_15min.pkl', 'rb')
T_apple_15min = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('/content/drive/My Drive/Tesis Konstanz/Clean data/t_test_sp_10min.pkl', 'rb')
T_sp_10min = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('/content/drive/My Drive/Tesis Konstanz/Clean data/t_test_sp_15min.pkl', 'rb')
T_sp_15min = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('/content/drive/My Drive/Tesis Konstanz/Clean data/t_test_dj_10min.pkl', 'rb')
T_dj_10min = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('/content/drive/My Drive/Tesis Konstanz/Clean data/t_test_dj_15min.pkl', 'rb')
T_dj_15min = pickle.load(pkl_file)
pkl_file.close()

## Descriptive stats Apple 10 and 15 min window

In [ ]:
#IForest threshold 10 minutes
X_train = pd.DataFrame(abs(T_sp_10min[1474:]))
X_test = pd.DataFrame(abs(T_dj_10min[1500:])) 

#Isolation forest training
if_mdlLst=alg.fit(X_train)
#iForest prediction
if_y_pred=if_mdlLst.predict(X_test)

#classified as -1 are anomalous  
print('The number of drift bursts in dow-jones are', sum(if_y_pred==-1))

#Threshold
thres = abs(X_test[if_y_pred==-1]) 
thres = thres.min()
print ('The initial threshold is', thres[0])

The number of drift bursts in dow-jones are 224
The initial threshold is 5.4678665239534325


In [ ]:
db_apple_10min = apple_sec[1479:-2][abs(T_apple_10min[1479:])>=thres[0]]
db_apple_10min['DRIFT_BURST'] = T_apple_10min[1479:][abs(T_apple_10min[1479:])>=thres[0]]
db_des_apple_10min = db_apple_10min.describe()
pd.options.display.float_format = lambda x : '{:.0f}'.format(x) if round(x,0) == x else '{:,.2f}'.format(x)
db_des_apple_10min.to_latex('/content/drive/My Drive/Tesis Konstanz/Clean data/tables/table_A1.tex')
db_des_apple_10min

,BID,ASK,ASKSIZ,BIDSIZ,SPREAD,MID_QUOTE,NUM_TICKS,DRIFT_BURST
count,190,190,190,190,190,190,190,190
mean,225.57,227.62,"1,293.67","2,334.30",2.05,226.59,81.67,-7.07
std,14.76,13.09,"3,724.62","5,728.68",1.87,13.92,42.73,1.08
min,198.96,201.00,11,13,0.15,199.98,7,-9.09
25%,210.57,215.01,329.50,356.50,0.46,212.66,47,-8.07
50%,229.96,230.53,556.50,807,1.02,230.25,73,-6.89
75%,239.65,240.04,1124,1836,3.50,239.84,115.75,-6.18
max,242.44,242.70,41898,49683,6.95,242.57,197,-5.47


In [ ]:
db_apple_10min

,BID,ASK,ASKSIZ,BIDSIZ,SPREAD,MID_QUOTE,NUM_TICKS,DRIFT_BURST
TIME_M,,,,,,,,
2010-05-06 14:36:56,242.44,242.70,2353,2305,0.26,242.57,115,-5.50
2010-05-06 14:36:57,242.24,242.68,516,1034,0.45,242.46,103,-5.68
2010-05-06 14:36:58,242.00,242.26,617,2696,0.26,242.13,95,-5.78
2010-05-06 14:36:59,241.86,242.09,237,541,0.23,241.98,62,-5.84
2010-05-06 14:37:00,241.64,242.07,1329,993,0.44,241.85,144,-5.90
...,...,...,...,...,...,...,...,...
2010-05-06 14:46:54,201.43,208.07,104,84,6.63,204.75,39,-7.15
2010-05-06 14:46:55,204.40,208.96,142,229,4.56,206.68,26,-6.59
2010-05-06 14:46:56,206.74,211.55,263,452,4.81,209.14,47,-6.63


In [ ]:
#IForest threshold 15 minutes
X_train = pd.DataFrame(abs(T_sp_15min[1474:]))
X_test = pd.DataFrame(abs(T_dj_15min[1500:])) 

#Isolation forest training
if_mdlLst=alg.fit(X_train)
#iForest prediction
if_y_pred=if_mdlLst.predict(X_test)

#classified as -1 are anomalous  
print('The number of drift bursts in dow-jones are', sum(if_y_pred==-1))

#Threshold
thres15 = abs(X_test[if_y_pred==-1]) 
thres15 = thres15.min()
print ('The initial threshold is', thres15[0])

The number of drift bursts in dow-jones are 293
The initial threshold is 5.924746965266589


In [ ]:
db_apple_15min = apple_sec[1479:-2][abs(T_apple_15min[1479:])>=thres15[0]]
db_apple_15min['DRIFT_BURST'] = T_apple_15min[1479:][abs(T_apple_15min[1479:])>=thres15[0]]
db_des_apple_15min = db_apple_15min.describe()
pd.options.display.float_format = lambda x : '{:.0f}'.format(x) if round(x,0) == x else '{:,.2f}'.format(x)
db_des_apple_15min.to_latex('/content/drive/My Drive/Tesis Konstanz/Clean data/tables/table_A2.tex')
db_des_apple_15min

,BID,ASK,ASKSIZ,BIDSIZ,SPREAD,MID_QUOTE,NUM_TICKS,DRIFT_BURST
count,177,177,177,177,177,177,177,177
mean,224.03,226.24,"1,351.53","2,403.27",2.21,225.13,78.18,-7.60
std,14.47,12.81,"3,856.34","5,923.76",1.89,13.64,42.25,1.21
min,198.96,201.00,11,13,0.21,199.98,7,-9.66
25%,209.70,213.81,313,356,0.51,211.82,44,-8.79
50%,228.30,229.89,560,800,1.56,229.23,70,-7.30
75%,238.62,239.06,1176,1715,3.70,238.84,110,-6.54
max,241.12,241.57,41898,49683,6.95,241.34,197,-5.93


In [ ]:
db_apple_15min

,BID,ASK,ASKSIZ,BIDSIZ,SPREAD,MID_QUOTE,NUM_TICKS,DRIFT_BURST
TIME_M,,,,,,,,
2010-05-06 14:37:07,241.12,241.57,1257,2049,0.46,241.34,167,-6.12
2010-05-06 14:37:08,240.75,241.29,916,1179,0.54,241.02,133,-6.20
2010-05-06 14:37:09,240.46,240.96,1232,4549,0.50,240.71,130,-6.19
2010-05-06 14:37:10,240.45,241.00,543,748,0.55,240.73,104,-6.15
2010-05-06 14:37:11,240.35,241.22,968,2136,0.87,240.78,123,-6.19
...,...,...,...,...,...,...,...,...
2010-05-06 14:46:55,204.40,208.96,142,229,4.56,206.68,26,-7.31
2010-05-06 14:46:56,206.74,211.55,263,452,4.81,209.14,47,-7.38
2010-05-06 14:46:57,207.17,210.80,43,433,3.63,208.98,25,-7.18


## Descriptive stats S&P 500 (SPY DTF) 10 and 15 min window

In [ ]:
db_sp_10min = s_p_sec[1474:-2][abs(T_sp_10min[1474:])>=thres[0]]
db_sp_15min = s_p_sec[1474:-2][abs(T_sp_15min[1474:])>=thres15[0]]

In [ ]:
db_sp_10min['DRIFT_BURST'] = T_sp_10min[1474:][abs(T_sp_10min[1474:])>=thres[0]]
db_des_sp_10min = db_sp_10min.describe()
pd.options.display.float_format = lambda x : '{:.0f}'.format(x) if round(x,0) == x else '{:,.2f}'.format(x)
db_des_sp_10min.to_latex('/content/drive/My Drive/Tesis Konstanz/Clean data/tables/table_A3.tex')
db_des_sp_10min

,BID,ASK,ASKSIZ,BIDSIZ,SPREAD,MID_QUOTE,NUM_TICKS,DRIFT_BURST
count,213,213,213,213,213,213,213,213
mean,109.16,109.24,"28,771.20",21150,0.08,109.20,368.26,-7.64
std,1.63,1.56,"27,325.54","18,107.29",0.10,1.59,125.40,1.29
min,106.34,106.60,99,80,0.02,106.47,20,-10.08
25%,107.48,107.56,8459,7129,0.03,107.51,310,-8.81
50%,109.76,109.79,22695,17634,0.03,109.78,395,-7.36
75%,110.45,110.48,42691,30182,0.11,110.46,454,-6.77
max,111.81,111.84,188686,106917,0.43,111.82,573,-5.48


In [ ]:
db_sp_10min

,BID,ASK,ASKSIZ,BIDSIZ,SPREAD,MID_QUOTE,NUM_TICKS,DRIFT_BURST
TIME_M,,,,,,,,
2010-05-06 14:40:14,111.81,111.84,38930,30661,0.03,111.82,434,-5.66
2010-05-06 14:40:15,111.73,111.76,66090,22836,0.03,111.75,434,-5.75
2010-05-06 14:40:16,111.64,111.67,23525,21515,0.03,111.65,502,-5.66
2010-05-06 14:40:17,111.68,111.70,26019,21140,0.02,111.69,333,-5.56
2010-05-06 14:40:18,111.70,111.73,31899,19595,0.03,111.72,359,-5.54
...,...,...,...,...,...,...,...,...
2010-05-06 14:46:18,107.20,107.48,5120,1523,0.28,107.34,244,-6.73
2010-05-06 14:46:19,107.21,107.38,4615,1697,0.18,107.29,247,-6.52
2010-05-06 14:46:20,107.38,107.56,2762,6891,0.18,107.47,367,-6.31


In [ ]:
db_sp_15min['DRIFT_BURST'] = T_sp_15min[1474:][abs(T_sp_15min[1474:])>=thres15[0]]
db_des_sp_15min = db_sp_15min.describe()
pd.options.display.float_format = lambda x : '{:.0f}'.format(x) if round(x,0) == x else '{:,.2f}'.format(x)
db_des_sp_15min.to_latex('/content/drive/My Drive/Tesis Konstanz/Clean data/tables/table_A4.tex')
db_des_sp_15min

,BID,ASK,ASKSIZ,BIDSIZ,SPREAD,MID_QUOTE,NUM_TICKS,DRIFT_BURST
count,214,214,214,214,214,214,214,214
mean,109.10,109.19,"28,566.23","21,407.41",0.09,109.14,367.55,-8.32
std,1.60,1.52,"27,293.27","18,508.13",0.10,1.56,126.84,1.36
min,106.34,106.60,99,80,0.02,106.47,20,-10.86
25%,107.48,107.56,"8,377.75","6,681.75",0.03,107.52,310.25,-9.50
50%,109.61,109.64,21623,17555,0.03,109.62,398,-8.04
75%,110.44,110.47,"42,887.50","32,235.25",0.11,110.46,449,-7.38
max,111.81,111.84,188686,106917,0.43,111.82,573,-5.93


In [ ]:
db_sp_15min

,BID,ASK,ASKSIZ,BIDSIZ,SPREAD,MID_QUOTE,NUM_TICKS,DRIFT_BURST
TIME_M,,,,,,,,
2010-05-06 14:40:14,111.81,111.84,38930,30661,0.03,111.82,434,-5.93
2010-05-06 14:40:15,111.73,111.76,66090,22836,0.03,111.75,434,-5.97
2010-05-06 14:42:54,111.39,111.42,42281,53940,0.03,111.41,404,-5.94
2010-05-06 14:42:55,111.36,111.38,43001,43227,0.02,111.37,364,-6.10
2010-05-06 14:42:56,111.30,111.33,54229,41813,0.02,111.31,449,-6.22
...,...,...,...,...,...,...,...,...
2010-05-06 14:46:21,107.44,107.52,4329,5001,0.08,107.48,261,-6.87
2010-05-06 14:46:22,107.49,107.80,4652,2711,0.31,107.65,218,-6.55
2010-05-06 14:46:23,107.59,107.80,6250,2951,0.21,107.70,216,-6.25


##Descriptive stats Dow Jones (DIA DTF) 10 and 15 min window

In [ ]:
db_dj_10min = dow_sec[1500:-2][abs(T_dj_10min[1500:])>=thres[0]]
db_dj_15min = dow_sec[1500:-2][abs(T_dj_15min[1500:])>=thres15[0]]

In [ ]:
db_dj_10min['DRIFT_BURST'] = T_dj_10min[1500:][abs(T_dj_10min[1500:])>=thres[0]]
db_des_dj_10min = db_dj_10min.describe()
pd.options.display.float_format = lambda x : '{:.0f}'.format(x) if round(x,0) == x else '{:,.2f}'.format(x)
db_des_dj_10min.to_latex('/content/drive/My Drive/Tesis Konstanz/Clean data/tables/table_A5.tex')
db_des_dj_10min

,BID,ASK,ASKSIZ,BIDSIZ,SPREAD,MID_QUOTE,NUM_TICKS,DRIFT_BURST
count,231,231,231,231,231,231,231,231
mean,241.43,241.67,"1,710.95","2,426.63",0.24,241.55,443.99,-6.87
std,1.91,1.85,"1,118.14","1,434.19",0.10,1.88,215.96,1.00
min,239.03,239.23,93,246,0.08,239.13,70,-9.02
25%,239.77,240.17,952.50,1397,0.17,239.98,284,-7.38
50%,240.37,240.65,1452,2213,0.23,240.50,403,-6.81
75%,243.58,243.81,2272,"3,099.50",0.29,243.70,571.50,-6.17
max,244.99,245.08,6005,7129,0.66,245.04,1228,-5.47


In [ ]:
db_dj_10min

,BID,ASK,ASKSIZ,BIDSIZ,SPREAD,MID_QUOTE,NUM_TICKS,DRIFT_BURST
TIME_M,,,,,,,,
2018-02-05 15:06:59,244.99,245.08,1893,3081,0.08,245.04,489,-5.49
2018-02-05 15:07:00,244.91,245.02,4016,4392,0.12,244.96,865,-5.57
2018-02-05 15:07:01,244.81,244.91,4836,5195,0.10,244.86,935,-5.53
2018-02-05 15:07:02,244.80,244.91,3753,3847,0.11,244.85,728,-5.50
2018-02-05 15:07:03,244.88,244.99,5466,4807,0.11,244.94,1062,-5.55
...,...,...,...,...,...,...,...,...
2018-02-05 15:11:58,239.72,240.19,206,364,0.47,239.95,154,-5.99
2018-02-05 15:11:59,239.92,240.23,286,272,0.31,240.07,181,-5.89
2018-02-05 15:12:00,239.99,240.41,373,320,0.42,240.20,259,-5.74


In [ ]:
db_dj_15min['DRIFT_BURST'] = T_dj_15min[1500:][abs(T_dj_15min[1500:])>=thres15[0]]
db_des_dj_15min = db_dj_15min.describe()
pd.options.display.float_format = lambda x : '{:.0f}'.format(x) if round(x,0) == x else '{:,.2f}'.format(x)
db_des_dj_15min.to_latex('/content/drive/My Drive/Tesis Konstanz/Clean data/tables/table_A6.tex')
db_des_dj_15min

,BID,ASK,ASKSIZ,BIDSIZ,SPREAD,MID_QUOTE,NUM_TICKS,DRIFT_BURST
count,293,293,293,293,293,293,293,293
mean,241.93,242.16,"1,710.36","2,359.13",0.23,242.04,437.16,-7.48
std,2.08,1.99,"1,179.72","1,479.49",0.13,2.03,231.32,1.14
min,239.03,239.23,38,23,0.08,239.13,16,-10.04
25%,239.96,240.26,895,1331,0.14,240.10,274,-8.21
50%,241.72,241.91,1448,2144,0.20,241.81,381,-7.45
75%,244.11,244.25,2284,3110,0.29,244.17,568,-6.43
max,245.25,245.33,6394,7284,1.00,245.29,1321,-5.92


In [ ]:
db_dj_15min

,BID,ASK,ASKSIZ,BIDSIZ,SPREAD,MID_QUOTE,NUM_TICKS,DRIFT_BURST
TIME_M,,,,,,,,
2018-02-05 15:06:55,245.25,245.33,2137,1779,0.09,245.29,327,-5.93
2018-02-05 15:06:56,245.16,245.24,3300,3432,0.08,245.20,590,-6.00
2018-02-05 15:06:57,245.06,245.17,1616,1633,0.11,245.11,351,-6.04
2018-02-05 15:06:58,245.03,245.17,2746,3357,0.14,245.10,506,-6.13
2018-02-05 15:06:59,244.99,245.08,1893,3081,0.08,245.04,489,-6.23
...,...,...,...,...,...,...,...,...
2018-02-05 15:12:07,240.24,240.97,359,103,0.73,240.61,63,-6.18
2018-02-05 15:12:08,240.39,240.84,130,170,0.44,240.61,92,-6.16
2018-02-05 15:12:09,240.33,240.85,40,23,0.52,240.59,16,-6.10
